In [ ]:
import pandas as pd
import time
from datetime import datetime
import cv2
import numpy as np
import os
import youtube_dl
# import moviepy

In [ ]:
def get_sec(time_str):
    """Get Seconds from time."""
    h, m, s = time_str.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

def listdir_nohidden(directory):
    filelist = os.listdir(directory)
    return [x for x in filelist
            if not (x.startswith('.'))]

In [ ]:
classifier_directory = os.getcwd()
parent_directory = classifier_directory[:-11] #exclude '/classifier'

pathway_to_clips = classifier_directory + '/movie_clips'
pathway_to_frames = classifier_directory + '/frames'

In [ ]:
parent_directory

In [ ]:
# os.chdir('/Users/max/Documents/Insight/TwoStepper/classifier')
df  = pd.read_csv('twostepper_input.csv')
url_unique_vals = df.URL.unique()
title_unique_vals = df.Title.unique()

# [df['URL'].str.contains(url_unique_vals[0])]
df[df['Title'].str.match(title_unique_vals[0])]

In [ ]:
os.chdir(classifier_directory)
try:
    if not os.path.exists('youtube_videos'):
        os.makedirs('youtube_videos')
    if not os.path.exists('movie_clips'):
        os.makedirs('movie_clips')
    if not os.path.exists('frames'):
        os.makedirs('frames')        
except OSError:
    print ('Error: Creating directory of data')

os.chdir(classifier_directory + '/youtube_videos')

for kk in range(len(title_unique_vals)):
    url_lookup = url_unique_vals[kk]
#     !youtube-dl -f 'bestvideo[height<=480, ext=mp4]' $url_lookup --id
    title = title_unique_vals[kk]
    
    dfsub = df[df['Title'].str.match(title_unique_vals[kk])]
    dfsub = dfsub.reset_index()

    for jj in range(len(dfsub)):
        start_time = get_sec(dfsub.start_time[jj])
        end_time = get_sec(dfsub.end_time[jj])
        
        #augmenter
        early_start_time = start_time - (0.5 * (end_time - start_time))
        late_end_time = end_time + (0.5 * (end_time - start_time))
        late_start_time = start_time + (0.25 * (end_time - start_time))
        early_end_time = end_time - (0.25 * (end_time - start_time))
        
        movie_name = dfsub.URL[jj][-11:] + '.mp4'
        movie_out_clip = dfsub.move[jj] + '.' + dfsub.URL[jj][-11:] + '__' + str(start_time) + '__' + str(end_time) + '.mov'
        movie_out_clip_long = dfsub.move[jj] + '.' + dfsub.URL[jj][-11:] + '__' + str(early_start_time) + '__' + str(late_end_time) + '.mov'
#         movie_out_clip_short = dfsub.move[jj] + '.' + dfsub.URL[jj][-11:] + '__' + str(late_start_time) + '__' + str(early_end_time) + '.mov'
        movie_out_clip_early = dfsub.move[jj] + '.' + dfsub.URL[jj][-11:] + '__' + str(early_start_time) + '__' + str(early_end_time) + '.mov'
        movie_out_clip_late = dfsub.move[jj] + '.' + dfsub.URL[jj][-11:] + '__' + str(late_start_time) + '__' + str(late_end_time) + '.mov'
        
        movie_out_name_and_path = classifier_directory + '/movie_clips/' + movie_out_clip
        movie_out_name_and_path_long = classifier_directory + '/movie_clips/' + movie_out_clip_long 
#         movie_out_name_and_path_short = classifier_directory + '/movie_clips/' + movie_out_clip_short
        movie_out_name_and_path_early = classifier_directory + '/movie_clips/' + movie_out_clip_early 
        movie_out_name_and_path_late = classifier_directory + '/movie_clips/' + movie_out_clip_late

        #original 
        !ffmpeg -ss $start_time -to $end_time -i $movie_name -c copy $movie_out_name_and_path 
        #augmented
        !ffmpeg -ss $early_start_time -to $early_end_time -i $movie_name -c copy $movie_out_name_and_path_early 
        !ffmpeg -ss $early_start_time -to $late_end_time -i $movie_name -c copy $movie_out_name_and_path_long
        !ffmpeg -ss $late_start_time -to $late_end_time -i $movie_name -c copy $movie_out_name_and_path_late

#         !ffmpeg -ss $late_start_time -to $early_end_time -i $movie_name -c copy $movie_out_name_and_path_short 


    
    

In [ ]:
os.chdir(pathway_to_clips)
clip_names = listdir_nohidden(pathway_to_clips)


for ii in range(len(clip_names)):
    #################### Setting up the file ################
    clip_name = clip_names[ii]
    vidcap = cv2.VideoCapture(pathway_to_clips + '/' + clip_names[ii])
    success,image = vidcap.read()
    
#     videoFile = 
    
    frame_dir = pathway_to_frames + '/' + clip_name[:-4]
    
    try:
        if not os.path.exists(frame_dir):
            os.makedirs(frame_dir)
    except OSError:
        print ('Error: Creating directory of data')

#     # #################### Setting up parameters ################

#     #OpenCV is notorious for not being able to good to 
#     # predict how many frames are in a video. The point here is just to 
#     # populate the "desired_frames" list for all the individual frames
#     # you'd like to capture. 

#     fps = vidcap.get(cv2.CAP_PROP_FPS)
#     est_video_length_secs = 20         # Round up if not sure.
#     est_tot_frames = est_video_length_secs * fps  # Sets an upper bound # of frames in video clip

#     n = 5                            # Desired interval of frames to include
#     desired_frames = n * np.arange(est_tot_frames) 


#     # #################### Initiate Process ################

#     for i in desired_frames:
#         vidcap.set(1,i-1)                      
#         success,image = vidcap.read(1)         # image is an array of array of [R,G,B] values
#         frameId = vidcap.get(1)                # The 0th frame is often a throw-away
#         frame_name_and_path = frame_dir + '/frame%d.jpg'
#         cv2.imwrite(frame_name_and_path % frameId, image)

#     vidcap.release()
#     cv2.destroyAllWindows()


    est_tot_frames = vidcap.get(cv2.CAP_PROP_FRAME_COUNT)  # Sets an upper bound # of frames in video clip
    desired_frames = np.round(np.arange(0,est_tot_frames,est_tot_frames/30)) #split the video into 30 frames


#################### Initiate Process ################

    for i in desired_frames:
        vidcap.set(1,i-1)                      
        success,image = vidcap.read(1)         # image is an array of array of [R,G,B] values
        frameId = vidcap.get(1)                # The 0th frame is often a throw-away
        cv2.imwrite(frame_dir + '/' + clip_name + 'frame%d.jpg' % frameId, image)
    vidcap.release()
    cv2.destroyAllWindows()